In [9]:
import pdfminer
import requests
from bs4 import BeautifulSoup
from pdfminer import pdfparser
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import string
from scipy.spatial.distance import cosine
import numpy as np
from nltk import PorterStemmer
from nltk import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
import lda

In [10]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = file(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,\
                                  password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [11]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams
from pdfminer.converter import  TextConverter # , XMLConverter, HTMLConverter
import urllib2
from urllib2 import Request
import datetime
import re

# Define a PDF parser function
def parsePDF(url):

    # Open the url provided as an argument to the function and read the content
    open = urllib2.urlopen(Request(url)).read()

    # Cast to StringIO object
    from StringIO import StringIO
    memory_file = StringIO(open)

    # Create a PDF parser object associated with the StringIO object
    parser = PDFParser(memory_file)

    # Create a PDF document object that stores the document structure
    document = PDFDocument(parser)

    # Define parameters to the PDF device objet 
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    codec = 'utf-8'

    # Create a PDF device object
    device = TextConverter(rsrcmgr, retstr, codec = codec, laparams = laparams)

    # Create a PDF interpreter object
    interpreter = PDFPageInterpreter(rsrcmgr, device)

    # Process each page contained in the document
    for page in PDFPage.create_pages(document):
        interpreter.process_page(page)
        data =  retstr.getvalue()
    return data

In [12]:
def get_text(url):
    '''Function to pull all papers (pdf) off the arxiv'''
    base_url = url
    r = requests.get(base_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    pdfs = soup.findAll(title = 'Download PDF')
    links = [str(pdf).split()[1].strip('href="') for pdf in pdfs]
    urls = ['http://arxiv.org'+ link for link in links]
    articles = []
    for url in urls:
        articles.append(parsePDF(url))
    return articles

In [13]:
#steph = convert_pdf_to_txt('/Users/Colin/Downloads/plms.pdf')
#physics1 = convert_pdf_to_txt('/Users/Colin/Downloads/physics1.pdf')
#shake = convert_pdf_to_txt('/Users/Colin/Downloads/shakespeare-tempest.pdf')
#steph2 = convert_pdf_to_txt('/Users/Colin/Downloads/twists_final.pdf')

In [14]:
text= []
%time text = get_text('http://arxiv.org/list/math.NT/1508?show=150')


CPU times: user 10min 29s, sys: 3.32 s, total: 10min 33s
Wall time: 13min 58s


In [15]:
#Try to get rid of weird unicode characters
S = set()
S.update(letter for letter in string.lowercase)
S.update(letter for letter in string.uppercase)
S.update(digit for digit in string.digits)

In [58]:
#text.append(steph)
#text.append(shake)

#f = lambda x: 1 if x in string.printable else 0
#goodwords = []
#words = ['abcd\xce\xc2fg', 'running', 'primes', 'groups']
#for i in range(len(text)):
 #   goodwords.append([word for word in text[i].split() if all([f(char) for char in word])])


new_text=[]
for i in range(len(text)):
    new_text.append([word for word in text[i].split() if (word[0] in S) and (word[-1] in S)\
                    and (len(word)>3)])
   
for i in range(len(new_text)):
    new_text[i] = ' '.join(new_text[i])

In [65]:
#Make stop list
tfidf = TfidfVectorizer(stop_words='english')
Stop = set()
Stop.update([word for word in tfidf.get_stop_words()])
Stop.update(['theorem', 'lemma', 'proof', 'sum', 'difference', \
             'product', 'multiple', 'let', 'group', 'prime', 'log', 'limit', 'cid', 'result'\
            'main', 'conjecture', 'case', 'suppose', 'function', 'assume', 'follows', \
            'given', 'define', 'note', 'defined', 'class', 'proposition', 'function', 'set', \
             'primes', 'numbers','form', 'integers', 'curves', 'real', 'denote', 'extensions', 'result'])
Stop = list(Stop)

In [66]:
#new_text.append(new_steph)
tfidf_math = TfidfVectorizer(max_features=100, stop_words=Stop, ngram_range=(1,1), decode_error='ignore')
M = tfidf_math.fit_transform(new_text)

In [63]:
def get_topics(url, num_topics):
    '''Input: URL containing links to each document (pdf) in the corpus (i.e. arxiv)
    Output: the num_topics most important topics from the corpus
    '''
    text = get_text(url)
    clean_text = clean_pdf_text(text)


    tfidf_math = TfidfVectorizer(max_features=100, stop_words=math_stop(), ngram_range=(1,1), decode_error='ignore')
    M = tfidf_math.fit_transform(clean_text)

    feature_names = tfidf_math.get_feature_names()
    nmf = NMF(n_components=10)
    nmf.fit(M)
    topics = []
    for topic_idx, topic in enumerate(nmf.components_):
        topics.append((" ".join([feature_names[i] for i in topic.argsort()[:-10 - 1:-1]])))
    return topics

def clean_pdf_text(text):
    '''Input: list of text documents, each element of the list is a single (long) string
    Output: slightly cleaned up version, try to get rid of strange characters and short 'words'
    '''
    S = set()
    S.update(letter for letter in string.lowercase)
    S.update(letter for letter in string.uppercase)
    S.update(digit for digit in string.digits)


    new_text=[]
    for i in range(len(text)):
        new_text.append([word.lower() for word in text[i].split() if (word[0] in S) and (word[-1] in S)\
                    and (len(word)>3)])

    for i in range(len(new_text)):
        new_text[i] = ' '.join(new_text[i])

    return new_text


def math_stop():
    '''Add math specific words to the standard stop list'''
    tfidf = TfidfVectorizer(stop_words='english')
    Stop = set()
    Stop.update([word for word in tfidf.get_stop_words()])
    Stop.update(['theorem', 'lemma', 'proof', 'sum', 'difference', \
                 'product', 'multiple', 'let', 'group', 'prime', 'log', 'limit', 'cid', 'result'\
                'main', 'conjecture', 'case', 'suppose', 'function', 'assume', 'follows', \
                'given', 'define', 'note', 'defined', 'class', 'proposition', 'function', 'set', \
                 'primes', 'numbers','form', 'integers', 'curves', 'real', 'result', 'denote', 'extensions'])
    return list(Stop)


In [72]:
#topics = get_topics('http://arxiv.org/list/math.NT/1508?show=150', 10)
for i in range(len(topics)):
    print topics[i]

    print '\n'

number integer positive natural denote using result second consider following


galois abelian subgroup extension extensions groups image isomorphism element local


zeta functions representation series integral formula type local remark deﬁned


quadratic forms ideal ring small rank polynomials number linear lattice


sequence deﬁned polynomials following positive denote measure holds prove representation


modulo coeﬃcients formula polynomial integer series number lattice order section


lattice measure dimension exists sets point points following space prove


bound 18 inequality integral obtain constant large following series small


modular adic forms series associated deﬁne space functions coeﬃcients deﬁned


curve degree rational point polynomial irreducible points polynomials linear rank




In [67]:
#Get some topics

feature_names = tfidf_math.get_feature_names()
nmf = NMF(n_components=10)
nmf.fit(M)
for topic_idx, topic in enumerate(nmf.components_):
    print("Topic #%d:" % topic_idx)
    print(" ".join([feature_names[i]
                    for i in topic.argsort()[:-10 - 1:-1]]))
    print '\n'

Topic #0:
number bound xn 18 inequality large obtain positive constant integer


Topic #1:
galois abelian subgroup extension groups isomorphism image element local elements


Topic #2:
zeta functions series xn integral formula type deﬁned local values


Topic #3:
quadratic forms ideal number ring small rank lattice consider polynomials


Topic #4:
sequence number deﬁned integer following positive polynomials prove formula holds


Topic #5:
modulo coeﬃcients formula integer number series lattice polynomial positive xn


Topic #6:
measure dimension lattice sets exists point points following space dimensional


Topic #7:
representation positive non points exists number point second irreducible image


Topic #8:
modular adic forms series deﬁne functions associated space coeﬃcients deﬁned


Topic #9:
curve degree polynomial rational polynomials point points irreducible rank linear




In [30]:
print len(text)
print len(new_text)

142
142


In [147]:
#Get similarities
N = M.todense()
distances = []
for ix in range(N.shape[0]):
    distances.append(cosine(N[-3], N[ix]))
print np.argsort(distances)[:5]
print np.around(sorted(distances)[:5],3)

[139  16  17  24  42]
[-0.     0.298  0.335  0.338  0.373]


In [97]:
Dists = np.zeros((N.shape[0], N.shape[0]))
for ix in range(len(Dists)):
    for jx in range(len(Dists)):
        Dists[ix, jx] = cosine(N[ix], N[jx])

In [103]:
def get_distances(M, index1, index2):
    N = M.todense()
    Dists = np.zeros((N.shape[0], N.shape[0]))
    for ix in range(len(Dists)):
        for jx in range(len(Dists)):
            Dists[ix, jx] = cosine(N[ix], N[jx])
    return Dists[index1, index2]

In [106]:
get_distances(M, 25, 25)

-2.2204460492503131e-16

In [32]:
countvec = CountVectorizer(decode_error='ignore', stop_words=Stop, max_features=5000)
CV = countvec.fit_transform(new_text)

In [33]:

vocab=tuple(countvec.vocabulary_)

In [34]:
lda_model = lda.LDA(n_topics=10, n_iter=1500)
lda_model.fit(CV)

<lda.lda.LDA instance at 0x110a836c8>

In [35]:
topic_word = lda_model.topic_word_  # model.components_ also works
n_top_words = 8
topic_words = []
for i, topic_dist in enumerate(topic_word):
    topic_words.append(np.array(vocab)[np.argsort(topic_dist)][:-n_top_words:-1])

In [36]:
for i in range(len(topic_words)):
    print ' '.join(topic_words[i])
    print '\n'

c4 alternating eigenvarieties rouen proven revisited kedk


ryan g0 enumerating em orthonormal shown hg


main publishing c4 trial 3α laurent peter


bp1q pδs positive lack decomposes characterize c4


ag point finally katherine dred maxϕ omitted


information signiﬁcantly northcott c4 fourth space quote


elnj q1 4k matsumoto ge derangements idea


improves daeyeoul infer hr pencil pk1 isometric


cycles extensions descending average frames lines exhibiting


p2z olya average u1 mi branched hσ2




In [37]:
doc_topic = lda_model.doc_topic_

In [38]:
topic_most_pr = []
for n in range(10):
    topic_most_pr.append(doc_topic[n].argmax())

for n in range(10):
    print "doc: {} topic: {}".format(n, str(topic_words[topic_most_pr[n]]))

doc: 0 topic: [u'c4' u'alternating' u'eigenvarieties' u'rouen' u'proven' u'revisited'
 u'kedk']
doc: 1 topic: [u'c4' u'alternating' u'eigenvarieties' u'rouen' u'proven' u'revisited'
 u'kedk']
doc: 2 topic: [u'p2z' u'olya' u'average' u'u1' u'mi' u'branched' u'h\u03c32']
doc: 3 topic: [u'ag' u'point' u'finally' u'katherine' u'dred' u'max\u03d5' u'omitted']
doc: 4 topic: [u'improves' u'daeyeoul' u'infer' u'hr' u'pencil' u'pk1' u'isometric']
doc: 5 topic: [u'improves' u'daeyeoul' u'infer' u'hr' u'pencil' u'pk1' u'isometric']
doc: 6 topic: [u'ryan' u'g0' u'enumerating' u'em' u'orthonormal' u'shown' u'hg']
doc: 7 topic: [u'c4' u'alternating' u'eigenvarieties' u'rouen' u'proven' u'revisited'
 u'kedk']
doc: 8 topic: [u'main' u'publishing' u'c4' u'trial' u'3\u03b1' u'laurent' u'peter']
doc: 9 topic: [u'ag' u'point' u'finally' u'katherine' u'dred' u'max\u03d5' u'omitted']


In [39]:
topic_most_pr

[0, 0, 9, 4, 7, 7, 1, 0, 2, 4]

In [40]:
topic_words[topic_most_pr[3]]

array([u'ag', u'point', u'finally', u'katherine', u'dred', u'max\u03d5',
       u'omitted'], 
      dtype='<U20')

In [155]:
base_url = 'http://arxiv.org/list/math.NT/recent'
r = requests.get(base_url)
soup = BeautifulSoup(r.text, 'html.parser')
titles = soup.findAll(class_="list-title")
title_list = []
for ix in range(len(titles)):
    title_list.append(titles[ix].text)


In [166]:
def get_best_titles(M, index):
    N=M.todense()
    Dists=np.zeros((N.shape[0], N.shape[0]))
    for ix in range(len(Dists)):
        for jx in range(len(Dists)):
            Dists[ix, jx]=cosine(N[ix], N[jx])
    distances = np.around(Dists[index],3)
    best_five = np.argsort(distances)[1:6]
    best_titles = [title_list[i] for i in best_five]
    return best_titles

In [167]:
get_best_titles(M, 5)

[u'\nTitle: On the universality of the Epstein zeta function\n',
 u'\nTitle: Combinatorial analysis of polynomial flatness problem, Mahler measure  and ergodic theory\n',
 u'\nTitle: Asymptotic bases and probabilistic representation functions\n',
 u'\nTitle: Pell Numbers whose Euler Function is a Pell Number\n',
 u'\nTitle: Representation of positive integers by the form $x^3+y^3+z^3-3xyz$\n']

In [156]:
def get_text2(url):
    '''Input a URL from the arxiv (page of a list of papers), return a list of
    parsed articles (list of strings)
    '''

    base_url = url
    r = requests.get(base_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    pdfs = soup.findAll(title = 'Download PDF')
    links = [str(pdf).split()[1].strip('href="') for pdf in pdfs]
    urls = ['http://arxiv.org'+ link for link in links]
    articles = []
    for url in urls:
        articles.append(parsePDF(url))

    titles = soup.findAll(class_="list-title")
    title_list = []
    for ix in range(len(titles)):
        title_list.append(titles[ix].text)

    return articles, title_list

In [157]:
text, title_list = get_text2('http://arxiv.org/list/math.NT/recent')

In [158]:
new_text=[]
for i in range(len(text)):
    new_text.append([word for word in text[i].split() if (word[0] in S) and (word[-1] in S)\
                    and (len(word)>3)])
   
for i in range(len(new_text)):
    new_text[i] = ' '.join(new_text[i])

In [160]:
tfidf_math = TfidfVectorizer(max_features=100, stop_words=Stop, ngram_range=(1,1), decode_error='ignore')
M = tfidf_math.fit_transform(new_text)

In [170]:
for title in get_best_titles(M,4):
    print title


Title: Backward Orbit Conjecture for the Powering Map over Global Fields


Title: Additive equations in dense variables, and truncated restriction  estimates


Title: On some upper bounds for the zeta-function and the Dirichlet divisor  problem


Title: The class number one problem for the real quadratic fields  $\mathbb{Q}\left(\sqrt{(an)^2+4a}\right)$


Title: A simple proof of Bourgain's theorem on the singularity of the spectrum  of Ornstein's maps

